In [2]:
## Subsample dataset for testing

import numpy as np
import random

split = "abstraction"

partition = "test"
data_dir = f"/mnt/nas2/GrimaRepo/araymond/3dshapes/shapes3d_{split}_{partition}_"


images = np.load(dadta_dir + "images.npz")
labels = np.load(data_dir + "labels.npz")

num_samples = len(images['arr_0'])//10
indices = np.random.choice(len(images['arr_0']), num_samples, replace=False)

new_images = images['arr_0'][indices]
new_labels = labels['arr_0'][indices]

images_file = data_dir + "images_subsample_for_evaluation_in_training.npz"
labels_file = data_dir + "labels_subsample_for_evaluation_in_training.npz"


In [2]:
np.savez(images_file, new_images)
np.savez(labels_file, new_labels)

In [4]:
print(len(new_images))

0


In [15]:

indices = np.random.choice(len(images['arr_0']), num_samples, replace=False)

new_images = images['arr_0'][indices]
new_labels = labels['arr_0'][indices]

images_file = data_dir + "images_subsample_for_evaluation_in_training.npz"
labels_file = data_dir + "labels_subsample_for_evaluation_in_training.npz"


## Get Features for split for ViT architectures

In [ ]:
import torch
from torchvision import models, transforms
from torch.utils.data import TensorDataset, DataLoader
from tqdm.notebook import tqdm
import numpy as np
# Create preprocessed model
import os

os.environ["TORCH_HOME"] = "~/storage/cache"
arch = "vit_b_16" # vit_b_16, vit_b_32, vit_l_16, vit_l_32
dataset = "idsprites"
splits = ['train','shape',"scale",'orientation','x','y']
constructors = {
    "vit_b_16": models.vit_b_16,
    "vit_b_32": models.vit_b_32,
    "vit_l_16": models.vit_l_16,
    "vit_l_32": models.vit_l_32   
}

model_weights ={
    "vit_b_16": models.ViT_B_16_Weights.IMAGENET1K_V1,
    "vit_b_32": models.ViT_B_32_Weights.IMAGENET1K_V1,
    "vit_l_16": models.ViT_L_16_Weights.IMAGENET1K_V1,
    "vit_l_32": models.ViT_L_32_Weights.IMAGENET1K_V1   
}

def get_images_path(dataset, feat_args):
    split = feat_args['split']
    arch = feat_args['arch']
    n_shapes = feat_args['n_shapes'] if 'n_shapes' in feat_args else None
    polation = feat_args['polation'] if 'polation' in feat_args else None
    
    if dataset == "shapes3d":
        return f"/mnt/nas2/GrimaRepo/araymond/shapes3d/shapes3d_abstraction_{split}_images.npz"
    elif dataset == "idsprites":
        return f"idsprites/idsprites_{polation}_{n_shapes}_images_{split}.npz"
    else:
        raise ValueError(f"Dataset {dataset} not supported!")

def get_output_path(dataset, feat_args):

    split = feat_args['split']
    arch = feat_args['arch']
    n_shapes = feat_args['n_shapes'] if 'n_shapes' in feat_args else None
    polation = feat_args['polation'] if 'polation' in feat_args else None
    
    if dataset == "shapes3d":
        return f"/mnt/nas2/GrimaRepo/araymond/shapes3d/shapes3d_abstraction_{split}_images_feats_{arch}.npz"
    elif dataset == "idsprites":
           return f"/mnt/nas2/GrimaRepo/araymond/idsprites/idsprites_{polation}_{n_shapes}_images_feats_{split}_{arch}.npz"
    else:
        raise ValueError(f"Dataset {dataset} not supported!")

feat_args = {'split': None, 'arch': None, 'polation': 'inter', 'shapes': None}

for split in tqdm(splits):
    feat_args['split'] = split
    
    for n_shapes in tqdm([14]):
        feat_args['n_shapes'] = n_shapes
        # data_dir = f"/mnt/nas2/GrimaRepo/araymond//shapes3d_abstraction_{split}_images.npz"
        data_dir = get_images_path(dataset, feat_args)
        images = np.load(data_dir)['arr_0']
        # Data
        tensor_images = torch.from_numpy(images)
        ds = TensorDataset(tensor_images)
        dl = DataLoader(ds, batch_size=512, shuffle=False)
        
        for arch in tqdm(["vit_b_16", "vit_b_32", "vit_l_16", "vit_l_32"]):
            print(f"Currently getting reps for arch: {arch}, Dataset: {n_shapes}, Split: {split}")
            feat_args['arch'] = arch
            # Load a pretrained Vision Transformer model
            vit_model = constructors[arch](weights=model_weights[arch]).cuda()
            # Get the preprocessing transforms for the model
            preprocess = model_weights[arch].transforms()
            # Run the image through the model
            vit_model.heads = torch.nn.Identity() # We want to extract features
            vit_model.eval()  # Set the model to evaluation mode
    
            result = []
            
            with torch.no_grad():
                for x in tqdm(dl):
                    x = x[0].cuda()
                    input_tensor = preprocess(x)  # Add batch dimension
                    output = vit_model(input_tensor)
                    result.append(output.cpu().detach())
            
            v = torch.vstack(result)    
            results_path = get_output_path(dataset, feat_args)
            np.savez(results_path, v.numpy())

/home/araymond/storage/pyenv/versions/3.10.14/envs/mini/lib/python3.10/site-packages/transformers/utils/hub.py:127: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
There was a problem when trying to write in your cache folder (/storage/cache). You should set the environment variable TRANSFORMERS_CACHE to a writable directory.


  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

Currently getting reps for arch: vit_b_16, Dataset: 14, Split: train


...

  0%|          | 0/79 [00:00<?, ?it/s]

Currently getting reps for arch: vit_b_32, Dataset: 14, Split: shape


  0%|          | 0/79 [00:00<?, ?it/s]

Currently getting reps for arch: vit_l_16, Dataset: 14, Split: shape


  0%|          | 0/79 [00:00<?, ?it/s]

In [2]:
import torch
from torchvision import models, transforms
from torch.utils.data import TensorDataset, DataLoader
from tqdm.notebook import tqdm
import numpy as np
from easydict import EasyDict as edict
# Create preprocessed model
import os
import argparse

parser = argparse.ArgumentParser(description="Example of argparse usage")

parser.add_argument('--arch', type=str, help='Architecture to extract features from')

#args = parser.parse_args()
args = edict()
args.arch = "vit_b_16"
os.environ["TORCH_HOME"] = "~/storage/cache"
#arch = "vit_b_16" # vit_b_16, vit_b_32, vit_l_16, vit_l_32
dataset = "idsprites"
splits = [ 'train','shape',"scale",
		'orientation','x','y']
constructors = {
    "vit_b_16": models.vit_b_16,
    "vit_b_32": models.vit_b_32,
    "vit_l_16": models.vit_l_16,
    "vit_l_32": models.vit_l_32   
}

model_weights ={
    "vit_b_16": models.ViT_B_16_Weights.IMAGENET1K_V1,
    "vit_b_32": models.ViT_B_32_Weights.IMAGENET1K_V1,
    "vit_l_16": models.ViT_L_16_Weights.IMAGENET1K_V1,
    "vit_l_32": models.ViT_L_32_Weights.IMAGENET1K_V1   
}


def get_output_path(dataset, feat_args):

    split = feat_args['split']
    arch = feat_args['arch']
    n_shapes = feat_args['n_shapes'] if 'n_shapes' in feat_args else None
    polation = feat_args['polation'] if 'polation' in feat_args else None
    
    if dataset == "shapes3d":
        return f"/mnt/nas2/GrimaRepo/araymond/shapes3d/shapes3d_abstraction_{split}_images_feats_{arch}.npz"
    elif dataset == "idsprites":
           return f"idsprites/idsprites_images_feats_{arch}.pth"
    else:
        raise ValueError(f"Dataset {dataset} not supported!")



# data_dir = f"/mnt/nas2/GrimaRepo/araymond//shapes3d_abstraction_{split}_images.npz"
data_dir = f"idsprites/idsprites.pth"

print("Loading dataset...", flush=True)
tensor_images = torch.load(data_dir)['images']
# Data
#tensor_images = torch.from_numpy(images)
ds = TensorDataset(tensor_images)
dl = DataLoader(ds, batch_size=512, shuffle=False)
arch = args.arch

# Load a pretrained Vision Transformer model
print(f"Loading pretrained model {args.arch}...", flush=True)
vit_model = constructors[arch](weights=model_weights[arch]).cuda()
# Get the preprocessing transforms for the model
preprocess = model_weights[arch].transforms()
# Run the image through the model
vit_model.heads = torch.nn.Identity() # We want to extract features
vit_model.eval()  # Set the model to evaluation mode

result = []

print("Obtaining representations...", flush=True)
with torch.no_grad():
    for x in tqdm(dl):
        print(x)
        x = x.cuda()
        input_tensor = preprocess(x)  # Add batch dimension
        output = vit_model(input_tensor)
        result.append(output.cpu().detach())

v = torch.vstack(result)    
results_path = get_output_path(dataset, feat_args)
torch.save(v, results_path)


Loading dataset...
Loading pretrained model vit_b_16...
Obtaining representations...


  0%|          | 0/4127 [00:00<?, ?it/s]

[tensor([[[[0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          ...,
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0]]],


        [[[0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          ...,
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0]]],


        [[[0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          ...,
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0]]],


        ...,


        [[[0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          ...,
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0]]],


        [[[0, 0, 0,  ..., 0, 0, 0],
        

AttributeError: 'list' object has no attribute 'cuda'

In [7]:
x[0].shape

torch.Size([512, 1, 64, 64])

In [6]:
import torch
import numpy as np
data = torch.load("idsprites/idsprites.pth")

In [7]:
new_data = dict()
ds_size = len(data['images'])

In [8]:
indices = np.array(list(range(1000)))

In [12]:
new_data['images'] = data['images'][indices]
new_data['latents'] = data['latents'][indices]
new_data['latent_ids'] = data['latents'][indices]
new_data['meta'] = data['meta']

In [14]:
torch.save(new_data, "idsprites/idsprites_test.pth")